In [ ]:
##################### بررسی خوشه بندی های مختلف بر روی متن اول(Aggregation.txt)


In [ ]:
###################با جابه جایی یکی از مراکز با نقاط داده k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link= ["http://cs.joensuu.fi/sipu/datasets/Aggregation.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels
k=7
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
#زمان پابان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.8946700507614214
F-measure: 0.0761402186204297
Execution time: 0.24234461784362793
Dataset with new labels:
      x1     x2  labels  new_labels
0  15.55  28.65       2           2
1  14.90  27.55       2           2
2  14.45  28.35       2           2
3  14.15  28.80       2           2
4  13.75  28.05       2           2


In [ ]:
###################### با جابه جایی دو تا از مراکز با نقاط داده k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/Aggregation.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels
k = 7
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.8946700507614214
F-measure: 0.0761402186204297
Execution time: 0.8209073543548584
Dataset with new labels:
      x1     x2  labels  new_labels
0  15.55  28.65       2           2
1  14.90  27.55       2           2
2  14.45  28.35       2           2
3  14.15  28.80       2           2
4  13.75  28.05       2           2


In [ ]:
############## با کرنل دو جمله ای k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import polynomial_kernel

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/Aggregation.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median_polynomial
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

k = 7
# زمان شروع
start_time = time.time()
centers, labels = k_median_polynomial(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

purity = purity_score(dataset)

# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)

    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

f_measure_value = f_measure(dataset)

# زمان پایان
end_time = time.time()
execution_time = end_time - start_time

print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.47588832487309646
F-measure: 0.08567743520452033
Execution time: 1.4684431552886963
Dataset with new labels:
      x1     x2  labels  new_labels
0  15.55  28.65       2           3
1  14.90  27.55       2           3
2  14.45  28.35       2           3
3  14.15  28.80       2           3
4  13.75  28.05       2           3


In [ ]:
##############    با کرنل گوسی k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import rbf_kernel
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/Aggregation.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data, centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = np.array([data[labels == i].median(axis=0) for i in range(k)])
        if np.allclose(centers, new_centers):
            break
        centers = new_centers
    return centers, labels
k = 7
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
        precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.8946700507614214
F-measure: 0.0761402186204297
Execution time: 0.04657554626464844
Dataset with new labels:
      x1     x2  labels  new_labels
0  15.55  28.65       2           2
1  14.90  27.55       2           2
2  14.45  28.35       2           2
3  14.15  28.80       2           2
4  13.75  28.05       2           2


In [ ]:
#################         الگوریتم خوشه بندی طیفی

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigsh
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/Aggregation.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تابع خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels
l=9
k = 7
# زمان شروع
start_time = time.time()
labels = spectral_clustering(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.9543147208121827
F-measure: 0.3281789783088576
Execution time: 0.24765944480895996
Dataset with new labels:
      x1     x2  labels  new_labels
0  15.55  28.65       2           7
1  14.90  27.55       2           7
2  14.45  28.35       2           7
3  14.15  28.80       2           7
4  13.75  28.05       2           7


In [ ]:
###############مقایسه الگوریتم ها

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import polynomial_kernel, rbf_kernel
from scipy.sparse.linalg import eigsh

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/Aggregation.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
k=7
l=9

# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels

# نسخه دوم k_median (با فواصل مشابه نسخه اول شما)
def k_median2(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels

# K-Median با کرنل چندجمله‌ای
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

# K-Median با کرنل گوسی
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data, centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = np.array([data[labels == i].median(axis=0) for i in range(k)])
        if np.allclose(centers, new_centers):
            break
        centers = new_centers
    return centers, labels

# خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels

# محاسبه Purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

# محاسبه F-measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

# اجرای الگوریتم‌ها
def run_algorithm(algorithm, dataset, k):
    start_time = time.time()
    if algorithm == spectral_clustering:
        labels = algorithm(dataset[["x1", "x2"]], k)
        centers = None
    else:
        centers, labels = algorithm(dataset[["x1", "x2"]], k)
    labels += 1
    dataset["new_labels"] = labels
    purity = purity_score(dataset)
    f_measure_value = f_measure(dataset)
    end_time = time.time()
    execution_time = end_time - start_time
    return purity, f_measure_value, execution_time, dataset

# نتایج
results = []
algorithms = [k_median, k_median2, spectral_clustering, k_median_polynomial, k_median_rbf]
for algorithm in algorithms:
    purity, f_measure_value, execution_time, dataset_with_labels = run_algorithm(algorithm, dataset.copy(), k)
    results.append({
        "Algorithm": algorithm.__name__,
        "Purity": purity,
        "F-measure": f_measure_value,
        "Execution Time": execution_time
    })

results_df = pd.DataFrame(results)

# مقایسه نتایج
print("مقایسه نتایج الگوریتم‌ها:")
print(results_df)

مقایسه نتایج الگوریتم‌ها:
             Algorithm    Purity  F-measure  Execution Time
0             k_median  0.894670   0.076140        0.024602
1            k_median2  0.894670   0.076140        0.476010
2  spectral_clustering  0.897208   0.209552        0.249812
3  k_median_polynomial  0.475888   0.085677        0.905868
4         k_median_rbf  0.894670   0.076140        0.043948


In [ ]:
############## مشاهدات برروی متن اول به این شکل است که خوشه بندی طیفی در این مثال از هر نظر بهتر عمل میکند

In [ ]:
########################## انجام الگوریتم ها در تکست دوم(Compound.txt)

In [ ]:
###################با جابه جایی یکی از مراکز با نقاط داده k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link= ["http://cs.joensuu.fi/sipu/datasets/Compound.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_media
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels
k=6
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
#زمان پابان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.87468671679198
F-measure: 0.1647940074906367
Execution time: 0.04902195930480957
Dataset with new labels:
      x1     x2  labels  new_labels
0  26.75  22.15       1           5
1  29.80  22.15       1           5
2  31.55  21.10       1           5
3  27.70  20.85       1           5
4  29.90  19.95       1           5


In [ ]:
###################### با جابه جایی دو تا از مراکز با نقاط داده k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/Compound.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels
k =6
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.87468671679198
F-measure: 0.1647940074906367
Execution time: 1.2920897006988525
Dataset with new labels:
      x1     x2  labels  new_labels
0  26.75  22.15       1           5
1  29.80  22.15       1           5
2  31.55  21.10       1           5
3  27.70  20.85       1           5
4  29.90  19.95       1           5


In [ ]:
#################با کرنل دو جمله ای k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import polynomial_kernel

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/Compound.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median_polynomial
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

k = 6
# زمان شروع
start_time = time.time()
centers, labels = k_median_polynomial(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

purity = purity_score(dataset)

# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)

    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

f_measure_value = f_measure(dataset)

# زمان پایان
end_time = time.time()
execution_time = end_time - start_time

print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.47869674185463656
F-measure: 0.04006410256410257
Execution time: 1.589979648590088
Dataset with new labels:
      x1     x2  labels  new_labels
0  26.75  22.15       1           1
1  29.80  22.15       1           1
2  31.55  21.10       1           1
3  27.70  20.85       1           1
4  29.90  19.95       1           1


In [ ]:
############## با کرنل گوسی k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import rbf_kernel
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/Compound.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data, centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = np.array([data[labels == i].median(axis=0) for i in range(k)])
        if np.allclose(centers, new_centers):
            break
        centers = new_centers
    return centers, labels
k =6
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
        precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.87468671679198
F-measure: 0.1647940074906367
Execution time: 0.04909372329711914
Dataset with new labels:
      x1     x2  labels  new_labels
0  26.75  22.15       1           5
1  29.80  22.15       1           5
2  31.55  21.10       1           5
3  27.70  20.85       1           5
4  29.90  19.95       1           5


In [ ]:
#################         الگوریتم خوشه بندی طیفی

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigsh
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/Compound.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تابع خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels
l=9
k =6
# زمان شروع
start_time = time.time()
labels = spectral_clustering(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.8571428571428571
F-measure: 0.7442548363975008
Execution time: 0.07467508316040039
Dataset with new labels:
      x1     x2  labels  new_labels
0  26.75  22.15       1           1
1  29.80  22.15       1           1
2  31.55  21.10       1           1
3  27.70  20.85       1           1
4  29.90  19.95       1           1


In [ ]:
###############مقایسه الگوریتم ها

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import polynomial_kernel, rbf_kernel
from scipy.sparse.linalg import eigsh

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/Compound.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
k=6
l=9

# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels

# نسخه دوم k_median (با فواصل مشابه نسخه اول شما)
def k_median2(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels

# K-Median با کرنل چندجمله‌ای
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

# K-Median با کرنل گوسی
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data, centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = np.array([data[labels == i].median(axis=0) for i in range(k)])
        if np.allclose(centers, new_centers):
            break
        centers = new_centers
    return centers, labels

# خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels

# محاسبه Purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

# محاسبه F-measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

# اجرای الگوریتم‌ها
def run_algorithm(algorithm, dataset, k):
    start_time = time.time()
    if algorithm == spectral_clustering:
        labels = algorithm(dataset[["x1", "x2"]], k)
        centers = None
    else:
        centers, labels = algorithm(dataset[["x1", "x2"]], k)
    labels += 1
    dataset["new_labels"] = labels
    purity = purity_score(dataset)
    f_measure_value = f_measure(dataset)
    end_time = time.time()
    execution_time = end_time - start_time
    return purity, f_measure_value, execution_time, dataset

# نتایج
results = []
algorithms = [k_median, k_median2, spectral_clustering, k_median_polynomial, k_median_rbf]
for algorithm in algorithms:
    purity, f_measure_value, execution_time, dataset_with_labels = run_algorithm(algorithm, dataset.copy(), k)
    results.append({
        "Algorithm": algorithm.__name__,
        "Purity": purity,
        "F-measure": f_measure_value,
        "Execution Time": execution_time
    })

results_df = pd.DataFrame(results)

# مقایسه نتایج
print("مقایسه نتایج الگوریتم‌ها:")
print(results_df)

مقایسه نتایج الگوریتم‌ها:
             Algorithm    Purity  F-measure  Execution Time
0             k_median  0.874687   0.164794        0.022899
1            k_median2  0.874687   0.164794        0.714261
2  spectral_clustering  0.904762   0.424102        0.324050
3  k_median_polynomial  0.478697   0.040064        2.392724
4         k_median_rbf  0.874687   0.164794        0.041273


In [ ]:
########################## انجام الگوریتم ها در تکست سوم(pathbased.txt)

In [ ]:
###################با جابه جایی یکی از مراکز با نقاط داده k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link= ["http://cs.joensuu.fi/sipu/datasets/pathbased.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_media
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels
k=3
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
#زمان پابان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.74
F-measure: 0.09641873278236915
Execution time: 0.04737448692321777
Dataset with new labels:
      x1    x2  labels  new_labels
0  11.25  5.05       1           1
1  10.95  4.70       1           1
2   9.85  5.80       1           1
3   9.80  5.75       1           1
4   9.15  6.80       1           1


In [ ]:
###################### با جابه جایی دو تا از مراکز با نقاط داده k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/pathbased.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels
k =3
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.74
F-measure: 0.09641873278236915
Execution time: 0.4539005756378174
Dataset with new labels:
      x1    x2  labels  new_labels
0  11.25  5.05       1           1
1  10.95  4.70       1           1
2   9.85  5.80       1           1
3   9.80  5.75       1           1
4   9.15  6.80       1           1


In [ ]:
#################با کرنل دو جمله ای k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import polynomial_kernel

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/pathbased.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median_polynomial
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

k = 3
# زمان شروع
start_time = time.time()
centers, labels = k_median_polynomial(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

purity = purity_score(dataset)

# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)

    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

f_measure_value = f_measure(dataset)

# زمان پایان
end_time = time.time()
execution_time = end_time - start_time

print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.36666666666666664
F-measure: 0.1628883291351805
Execution time: 0.03727149963378906
Dataset with new labels:
      x1    x2  labels  new_labels
0  11.25  5.05       1           3
1  10.95  4.70       1           3
2   9.85  5.80       1           3
3   9.80  5.75       1           3
4   9.15  6.80       1           3


In [ ]:
#################با کرنل  گوسی k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import rbf_kernel
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/pathbased.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data, centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = np.array([data[labels == i].median(axis=0) for i in range(k)])
        if np.allclose(centers, new_centers):
            break
        centers = new_centers
    return centers, labels
k =3
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
        precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())


Purity: 0.74
F-measure: 0.09641873278236915
Execution time: 0.02574944496154785
Dataset with new labels:
      x1    x2  labels  new_labels
0  11.25  5.05       1           1
1  10.95  4.70       1           1
2   9.85  5.80       1           1
3   9.80  5.75       1           1
4   9.15  6.80       1           1


In [ ]:
#################         الگوریتم خوشه بندی طیفی

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigsh
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/pathbased.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تابع خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels
l=9
k =3
# زمان شروع
start_time = time.time()
labels = spectral_clustering(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.59
F-measure: 0.49045527842808384
Execution time: 0.05619072914123535
Dataset with new labels:
      x1    x2  labels  new_labels
0  11.25  5.05       1           1
1  10.95  4.70       1           1
2   9.85  5.80       1           1
3   9.80  5.75       1           1
4   9.15  6.80       1           1


In [ ]:
###############مقایسه الگوریتم ها

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import polynomial_kernel, rbf_kernel
from scipy.sparse.linalg import eigsh

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/pathbased.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
k=3
l=9

# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels

# نسخه دوم k_median (با فواصل مشابه نسخه اول شما)
def k_median2(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels

# K-Median با کرنل چندجمله‌ای
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

# K-Median با کرنل گوسی
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data, centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = np.array([data[labels == i].median(axis=0) for i in range(k)])
        if np.allclose(centers, new_centers):
            break
        centers = new_centers
    return centers, labels

# خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels

# محاسبه Purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

# محاسبه F-measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

# اجرای الگوریتم‌ها
def run_algorithm(algorithm, dataset, k):
    start_time = time.time()
    if algorithm == spectral_clustering:
        labels = algorithm(dataset[["x1", "x2"]], k)
        centers = None
    else:
        centers, labels = algorithm(dataset[["x1", "x2"]], k)
    labels += 1
    dataset["new_labels"] = labels
    purity = purity_score(dataset)
    f_measure_value = f_measure(dataset)
    end_time = time.time()
    execution_time = end_time - start_time
    return purity, f_measure_value, execution_time, dataset

# نتایج
results = []
algorithms = [k_median, k_median2, spectral_clustering, k_median_polynomial, k_median_rbf]
for algorithm in algorithms:
    purity, f_measure_value, execution_time, dataset_with_labels = run_algorithm(algorithm, dataset.copy(), k)
    results.append({
        "Algorithm": algorithm.__name__,
        "Purity": purity,
        "F-measure": f_measure_value,
        "Execution Time": execution_time
    })

results_df = pd.DataFrame(results)

# مقایسه نتایج
print("مقایسه نتایج الگوریتم‌ها:")
print(results_df)

مقایسه نتایج الگوریتم‌ها:
             Algorithm    Purity  F-measure  Execution Time
0             k_median  0.740000   0.096419        0.012383
1            k_median2  0.740000   0.096419        0.218505
2  spectral_clustering  0.586667   0.436108        0.028841
3  k_median_polynomial  0.366667   0.162888        0.011394
4         k_median_rbf  0.740000   0.096419        0.041589


In [ ]:
########################## انجام الگوریتم ها در تکست چهارم(spiral.txt)

In [ ]:
###################با جابه جایی یکی از مراکز با نقاط داده k_median

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link= ["http://cs.joensuu.fi/sipu/datasets/spiral.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_media
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels
k=3
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
#زمان پابان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.3557692307692308
F-measure: 0.3275272872589949
Execution time: 0.010505914688110352
Dataset with new labels:
      x1    x2  labels  new_labels
0  31.95  7.95       3           2
1  31.15  7.30       3           2
2  30.45  6.65       3           2
3  29.70  6.00       3           2
4  28.90  5.55       3           2


In [ ]:
###################### با جابه جایی دو تا از مراکز با نقاط داده k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/spiral.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels
k =3
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.3557692307692308
F-measure: 0.3275272872589949
Execution time: 0.7037723064422607
Dataset with new labels:
      x1    x2  labels  new_labels
0  31.95  7.95       3           2
1  31.15  7.30       3           2
2  30.45  6.65       3           2
3  29.70  6.00       3           2
4  28.90  5.55       3           2


In [ ]:
#################با کرنل دو جمله ای k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import polynomial_kernel

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/spiral.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median_polynomial
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

k = 3
# زمان شروع
start_time = time.time()
centers, labels = k_median_polynomial(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

purity = purity_score(dataset)

# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)

    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

f_measure_value = f_measure(dataset)

# زمان پایان
end_time = time.time()
execution_time = end_time - start_time

print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.34615384615384615
F-measure: 0.2775933272126493
Execution time: 0.6780035495758057
Dataset with new labels:
      x1    x2  labels  new_labels
0  31.95  7.95       3           3
1  31.15  7.30       3           3
2  30.45  6.65       3           3
3  29.70  6.00       3           3
4  28.90  5.55       3           3


In [ ]:
#################با کرنل  گوسی k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import rbf_kernel
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/spiral.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data, centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = np.array([data[labels == i].median(axis=0) for i in range(k)])
        if np.allclose(centers, new_centers):
            break
        centers = new_centers
    return centers, labels
k =3
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
        precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.3557692307692308
F-measure: 0.3275272872589949
Execution time: 0.043892621994018555
Dataset with new labels:
      x1    x2  labels  new_labels
0  31.95  7.95       3           2
1  31.15  7.30       3           2
2  30.45  6.65       3           2
3  29.70  6.00       3           2
4  28.90  5.55       3           2


In [ ]:
#################         الگوریتم خوشه بندی طیفی

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigsh
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/spiral.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تابع خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels
l=9
k =3
# زمان شروع
start_time = time.time()
labels = spectral_clustering(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.6121794871794872
F-measure: 0.19995040176152726
Execution time: 0.3169400691986084
Dataset with new labels:
      x1    x2  labels  new_labels
0  31.95  7.95       3           2
1  31.15  7.30       3           2
2  30.45  6.65       3           2
3  29.70  6.00       3           2
4  28.90  5.55       3           2


In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import polynomial_kernel, rbf_kernel
from scipy.sparse.linalg import eigsh

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/spiral.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
k=3
l=9

# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels

# نسخه دوم k_median (با فواصل مشابه نسخه اول شما)
def k_median2(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels

# K-Median با کرنل چندجمله‌ای
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

# K-Median با کرنل گوسی
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data, centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = np.array([data[labels == i].median(axis=0) for i in range(k)])
        if np.allclose(centers, new_centers):
            break
        centers = new_centers
    return centers, labels

# خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels

# محاسبه Purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

# محاسبه F-measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

# اجرای الگوریتم‌ها
def run_algorithm(algorithm, dataset, k):
    start_time = time.time()
    if algorithm == spectral_clustering:
        labels = algorithm(dataset[["x1", "x2"]], k)
        centers = None
    else:
        centers, labels = algorithm(dataset[["x1", "x2"]], k)
    labels += 1
    dataset["new_labels"] = labels
    purity = purity_score(dataset)
    f_measure_value = f_measure(dataset)
    end_time = time.time()
    execution_time = end_time - start_time
    return purity, f_measure_value, execution_time, dataset

# نتایج
results = []
algorithms = [k_median, k_median2, spectral_clustering, k_median_polynomial, k_median_rbf]
for algorithm in algorithms:
    purity, f_measure_value, execution_time, dataset_with_labels = run_algorithm(algorithm, dataset.copy(), k)
    results.append({
        "Algorithm": algorithm.__name__,
        "Purity": purity,
        "F-measure": f_measure_value,
        "Execution Time": execution_time
    })

results_df = pd.DataFrame(results)

# مقایسه نتایج
print("مقایسه نتایج الگوریتم‌ها:")
print(results_df)

مقایسه نتایج الگوریتم‌ها:
             Algorithm    Purity  F-measure  Execution Time
0             k_median  0.355769   0.327527        0.030015
1            k_median2  0.355769   0.327527        0.399443
2  spectral_clustering  0.657051   0.079365        0.081156
3  k_median_polynomial  0.346154   0.277593        0.799428
4         k_median_rbf  0.355769   0.327527        0.030217


In [ ]:
########################انجام الگوریتم ها در تکست پنجم(D31.txt)

In [ ]:
###################با جابه جایی یکی از مراکز با نقاط داده k_median

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link= ["http://cs.joensuu.fi/sipu/datasets/D31.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_media
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels
k=31
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
#زمان پابان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.7893548387096774
F-measure: 0.0008378718056137411
Execution time: 0.30080747604370117
Dataset with new labels:
        x1      x2  labels  new_labels
0  25.0514  5.7475       1          14
1  26.6614  7.3414       1          30
2  25.2653  6.2466       1          14
3  25.2285  4.7447       1          14
4  25.7529  5.1564       1          14


In [ ]:
###################### با جابه جایی دو تا از مراکز با نقاط داده k_median محاسبه

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/D31.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels
k =31
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.7893548387096774
F-measure: 0.0008378718056137411
Execution time: 6.320572137832642
Dataset with new labels:
        x1      x2  labels  new_labels
0  25.0514  5.7475       1          14
1  26.6614  7.3414       1          30
2  25.2653  6.2466       1          14
3  25.2285  4.7447       1          14
4  25.7529  5.1564       1          14


In [ ]:
#################با کرنل دو جمله ای k_median محاسبه

In [111]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import polynomial_kernel

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/D31.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median_polynomial
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

k = 31
# زمان شروع
start_time = time.time()
centers, labels = k_median_polynomial(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

purity = purity_score(dataset)

# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)

    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

f_measure_value = f_measure(dataset)

# زمان پایان
end_time = time.time()
execution_time = end_time - start_time

print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.06451612903225806
F-measure: 0.00019424210891432535
Execution time: 5.582829236984253
Dataset with new labels:
        x1      x2  labels  new_labels
0  25.0514  5.7475       1          10
1  26.6614  7.3414       1          10
2  25.2653  6.2466       1          10
3  25.2285  4.7447       1          10
4  25.7529  5.1564       1          10


In [ ]:
#################با کرنل  گوسی k_median محاسبه

In [84]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import rbf_kernel

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/D31.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)

# تعریف تابع k_median
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data.to_numpy(), centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

k = 31

# زمان شروع
start_time = time.time()
centers, labels = k_median_rbf(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels

# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

purity = purity_score(dataset)

# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = dataset["new_labels"].to_numpy()
        tp = np.sum((true_labels == i + 1) & (predicted_labels == i + 1))
        fp = np.sum((true_labels != i + 1) & (predicted_labels == i + 1))
        fn = np.sum((true_labels == i + 1) & (predicted_labels != i + 1))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

f_measure_value = f_measure(dataset)

# زمان پایان
end_time = time.time()
execution_time = end_time - start_time

print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.7893548387096774
F-measure: 0.0008378718056137411
Execution time: 0.46477818489074707
Dataset with new labels:
        x1      x2  labels  new_labels
0  25.0514  5.7475       1          14
1  26.6614  7.3414       1          30
2  25.2653  6.2466       1          14
3  25.2285  4.7447       1          14
4  25.7529  5.1564       1          14


In [ ]:
#################         الگوریتم خوشه بندی طیفی

In [85]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigsh
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/D31.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تابع خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels
l=9
k =31
# زمان شروع
start_time = time.time()
labels = spectral_clustering(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.6083870967741936
F-measure: 0.06665165055605297
Execution time: 90.07427644729614
Dataset with new labels:
        x1      x2  labels  new_labels
0  25.0514  5.7475       1           2
1  26.6614  7.3414       1           2
2  25.2653  6.2466       1           2
3  25.2285  4.7447       1           2
4  25.7529  5.1564       1           2


In [112]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import polynomial_kernel, rbf_kernel
from scipy.sparse.linalg import eigsh

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/D31.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
k=31
l=9

# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels

# نسخه دوم k_median (با فواصل مشابه نسخه اول شما)
def k_median2(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels

# K-Median با کرنل چندجمله‌ای
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

# K-Median با کرنل گوسی
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data, centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = np.array([data[labels == i].median(axis=0) for i in range(k)])
        if np.allclose(centers, new_centers):
            break
        centers = new_centers
    return centers, labels

# خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels

# محاسبه Purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

# محاسبه F-measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

# اجرای الگوریتم‌ها
def run_algorithm(algorithm, dataset, k):
    start_time = time.time()
    if algorithm == spectral_clustering:
        labels = algorithm(dataset[["x1", "x2"]], k)
        centers = None
    else:
        centers, labels = algorithm(dataset[["x1", "x2"]], k)
    labels += 1
    dataset["new_labels"] = labels
    purity = purity_score(dataset)
    f_measure_value = f_measure(dataset)
    end_time = time.time()
    execution_time = end_time - start_time
    return purity, f_measure_value, execution_time, dataset

# نتایج
results = []
algorithms = [k_median, k_median2, spectral_clustering, k_median_polynomial, k_median_rbf]
for algorithm in algorithms:
    purity, f_measure_value, execution_time, dataset_with_labels = run_algorithm(algorithm, dataset.copy(), k)
    results.append({
        "Algorithm": algorithm.__name__,
        "Purity": purity,
        "F-measure": f_measure_value,
        "Execution Time": execution_time
    })

results_df = pd.DataFrame(results)

# مقایسه نتایج
print("مقایسه نتایج الگوریتم‌ها:")
print(results_df)

مقایسه نتایج الگوریتم‌ها:
             Algorithm    Purity  F-measure  Execution Time
0             k_median  0.789355   0.000838        0.171317
1            k_median2  0.789355   0.000838        3.196847
2  spectral_clustering  0.577097   0.089400       71.564161
3  k_median_polynomial  0.064516   0.000194        5.688468
4         k_median_rbf  0.789355   0.000838        0.165985


In [ ]:
########################انجام الگوریتم ها در تکست ششم(R15.txt)

In [ ]:
###################با جابه جایی یکی از مراکز با نقاط داده k_median

In [91]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link= ["http://cs.joensuu.fi/sipu/datasets/R15.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_media
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels
k=15
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
#زمان پابان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.8616666666666667
F-measure: 0.24399105163921148
Execution time: 0.08253240585327148
Dataset with new labels:
       x1      x2  labels  new_labels
0   9.802  10.132       1           1
1  10.350   9.768       1           1
2  10.098   9.988       1           1
3   9.730   9.910       1           1
4   9.754  10.430       1           1


In [ ]:
###################### با جابه جایی دو تا از مراکز با نقاط داده k_median محاسبه

In [93]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/R15.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels
k =15
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.8616666666666667
F-measure: 0.24399105163921148
Execution time: 2.2917191982269287
Dataset with new labels:
       x1      x2  labels  new_labels
0   9.802  10.132       1           1
1  10.350   9.768       1           1
2  10.098   9.988       1           1
3   9.730   9.910       1           1
4   9.754  10.430       1           1


In [ ]:
#################با کرنل دو جمله ای k_median محاسبه

In [89]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import polynomial_kernel

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/R15.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median_polynomial
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

k = 15
# زمان شروع
start_time = time.time()
centers, labels = k_median_polynomial(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

purity = purity_score(dataset)

# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)

    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

f_measure_value = f_measure(dataset)

# زمان پایان
end_time = time.time()
execution_time = end_time - start_time

print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.09166666666666666
F-measure: 0.008533333333333334
Execution time: 3.134512424468994
Dataset with new labels:
       x1      x2  labels  new_labels
0   9.802  10.132       1           1
1  10.350   9.768       1           1
2  10.098   9.988       1           1
3   9.730   9.910       1           1
4   9.754  10.430       1           1


In [ ]:
#################با کرنل  گوسی k_median محاسبه

In [88]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import rbf_kernel

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/R15.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)

# تعریف تابع k_median
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data.to_numpy(), centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

k = 15

# زمان شروع
start_time = time.time()
centers, labels = k_median_rbf(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels

# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

purity = purity_score(dataset)

# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = dataset["new_labels"].to_numpy()
        tp = np.sum((true_labels == i + 1) & (predicted_labels == i + 1))
        fp = np.sum((true_labels != i + 1) & (predicted_labels == i + 1))
        fn = np.sum((true_labels == i + 1) & (predicted_labels != i + 1))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

f_measure_value = f_measure(dataset)

# زمان پایان
end_time = time.time()
execution_time = end_time - start_time

print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.8616666666666667
F-measure: 0.24399105163921148
Execution time: 0.19124841690063477
Dataset with new labels:
       x1      x2  labels  new_labels
0   9.802  10.132       1           1
1  10.350   9.768       1           1
2  10.098   9.988       1           1
3   9.730   9.910       1           1
4   9.754  10.430       1           1


In [ ]:
#################         الگوریتم خوشه بندی طیفی

In [92]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigsh
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/R15.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تابع خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels
l=9
k =15
# زمان شروع
start_time = time.time()
labels = spectral_clustering(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.7316666666666667
F-measure: 0.022408963585434174
Execution time: 1.6195805072784424
Dataset with new labels:
       x1      x2  labels  new_labels
0   9.802  10.132       1          14
1  10.350   9.768       1          14
2  10.098   9.988       1          14
3   9.730   9.910       1          14
4   9.754  10.430       1          14


In [ ]:
###مقایسه الگوریتم ها

In [95]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import polynomial_kernel, rbf_kernel
from scipy.sparse.linalg import eigsh

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/R15.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
k=15
l=9

# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels

# نسخه دوم k_median (با فواصل مشابه نسخه اول شما)
def k_median2(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels

# K-Median با کرنل چندجمله‌ای
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

# K-Median با کرنل گوسی
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data, centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = np.array([data[labels == i].median(axis=0) for i in range(k)])
        if np.allclose(centers, new_centers):
            break
        centers = new_centers
    return centers, labels

# خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels

# محاسبه Purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

# محاسبه F-measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

# اجرای الگوریتم‌ها
def run_algorithm(algorithm, dataset, k):
    start_time = time.time()
    if algorithm == spectral_clustering:
        labels = algorithm(dataset[["x1", "x2"]], k)
        centers = None
    else:
        centers, labels = algorithm(dataset[["x1", "x2"]], k)
    labels += 1
    dataset["new_labels"] = labels
    purity = purity_score(dataset)
    f_measure_value = f_measure(dataset)
    end_time = time.time()
    execution_time = end_time - start_time
    return purity, f_measure_value, execution_time, dataset

# نتایج
results = []
algorithms = [k_median, k_median2, spectral_clustering, k_median_polynomial, k_median_rbf]
for algorithm in algorithms:
    purity, f_measure_value, execution_time, dataset_with_labels = run_algorithm(algorithm, dataset.copy(), k)
    results.append({
        "Algorithm": algorithm.__name__,
        "Purity": purity,
        "F-measure": f_measure_value,
        "Execution Time": execution_time
    })

results_df = pd.DataFrame(results)

# مقایسه نتایج
print("مقایسه نتایج الگوریتم‌ها:")
print(results_df)

مقایسه نتایج الگوریتم‌ها:
             Algorithm    Purity  F-measure  Execution Time
0             k_median  0.861667   0.243991        0.071751
1            k_median2  0.861667   0.243991        1.559398
2  spectral_clustering  0.918333   0.065823        0.366690
3  k_median_polynomial  0.091667   0.008533        1.814952
4         k_median_rbf  0.861667   0.243991        0.086007


In [ ]:
########################## انجام الگوریتم ها در تکست هفتم(jain.txt)

In [ ]:
###################با جابه جایی یکی از مراکز با نقاط داده k_median

In [101]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link= ["http://cs.joensuu.fi/sipu/datasets/jain.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_media
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels
k=2
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
#زمان پابان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.7533512064343163
F-measure: 0.19784946236559142
Execution time: 0.024845600128173828
Dataset with new labels:
     x1     x2  labels  new_labels
0  0.85  17.45       2           2
1  0.75  15.60       2           2
2  3.30  15.45       2           2
3  5.25  14.20       2           2
4  4.90  15.65       2           2


In [ ]:
###################### با جابه جایی دو تا از مراکز با نقاط داده k_median محاسبه

In [100]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/jain.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels
k =2
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.7533512064343163
F-measure: 0.19784946236559142
Execution time: 0.4446718692779541
Dataset with new labels:
     x1     x2  labels  new_labels
0  0.85  17.45       2           2
1  0.75  15.60       2           2
2  3.30  15.45       2           2
3  5.25  14.20       2           2
4  4.90  15.65       2           2


In [ ]:
#################با کرنل دو جمله ای k_median محاسبه

In [99]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import polynomial_kernel

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/jain.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median_polynomial
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

k = 2
# زمان شروع
start_time = time.time()
centers, labels = k_median_polynomial(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

purity = purity_score(dataset)

# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)

    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

f_measure_value = f_measure(dataset)

# زمان پایان
end_time = time.time()
execution_time = end_time - start_time

print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.9168900804289544
F-measure: 0.10828034816547491
Execution time: 0.024169445037841797
Dataset with new labels:
     x1     x2  labels  new_labels
0  0.85  17.45       2           2
1  0.75  15.60       2           2
2  3.30  15.45       2           2
3  5.25  14.20       2           2
4  4.90  15.65       2           2


In [ ]:
#################با کرنل  گوسی k_median محاسبه

In [98]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import rbf_kernel

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/jain.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)

# تعریف تابع k_median
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data.to_numpy(), centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

k = 2

# زمان شروع
start_time = time.time()
centers, labels = k_median_rbf(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels

# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

purity = purity_score(dataset)

# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = dataset["new_labels"].to_numpy()
        tp = np.sum((true_labels == i + 1) & (predicted_labels == i + 1))
        fp = np.sum((true_labels != i + 1) & (predicted_labels == i + 1))
        fn = np.sum((true_labels == i + 1) & (predicted_labels != i + 1))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

f_measure_value = f_measure(dataset)

# زمان پایان
end_time = time.time()
execution_time = end_time - start_time

print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.7533512064343163
F-measure: 0.7931225734886301
Execution time: 0.020768165588378906
Dataset with new labels:
     x1     x2  labels  new_labels
0  0.85  17.45       2           2
1  0.75  15.60       2           2
2  3.30  15.45       2           2
3  5.25  14.20       2           2
4  4.90  15.65       2           2


In [ ]:
#################         الگوریتم خوشه بندی طیفی

In [97]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigsh
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/jain.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تابع خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels
l=9
k =2
# زمان شروع
start_time = time.time()
labels = spectral_clustering(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.739946380697051
F-measure: 0.6843275797594913
Execution time: 0.28987550735473633
Dataset with new labels:
     x1     x2  labels  new_labels
0  0.85  17.45       2           1
1  0.75  15.60       2           1
2  3.30  15.45       2           1
3  5.25  14.20       2           1
4  4.90  15.65       2           1


In [ ]:
###مفایسه الگوریتم

In [96]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import polynomial_kernel, rbf_kernel
from scipy.sparse.linalg import eigsh

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/jain.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
k=2
l=9

# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels

# نسخه دوم k_median (با فواصل مشابه نسخه اول شما)
def k_median2(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels

# K-Median با کرنل چندجمله‌ای
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

# K-Median با کرنل گوسی
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data, centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = np.array([data[labels == i].median(axis=0) for i in range(k)])
        if np.allclose(centers, new_centers):
            break
        centers = new_centers
    return centers, labels

# خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels

# محاسبه Purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

# محاسبه F-measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

# اجرای الگوریتم‌ها
def run_algorithm(algorithm, dataset, k):
    start_time = time.time()
    if algorithm == spectral_clustering:
        labels = algorithm(dataset[["x1", "x2"]], k)
        centers = None
    else:
        centers, labels = algorithm(dataset[["x1", "x2"]], k)
    labels += 1
    dataset["new_labels"] = labels
    purity = purity_score(dataset)
    f_measure_value = f_measure(dataset)
    end_time = time.time()
    execution_time = end_time - start_time
    return purity, f_measure_value, execution_time, dataset

# نتایج
results = []
algorithms = [k_median, k_median2, spectral_clustering, k_median_polynomial, k_median_rbf]
for algorithm in algorithms:
    purity, f_measure_value, execution_time, dataset_with_labels = run_algorithm(algorithm, dataset.copy(), k)
    results.append({
        "Algorithm": algorithm.__name__,
        "Purity": purity,
        "F-measure": f_measure_value,
        "Execution Time": execution_time
    })

results_df = pd.DataFrame(results)

# مقایسه نتایج
print("مقایسه نتایج الگوریتم‌ها:")
print(results_df)

مقایسه نتایج الگوریتم‌ها:
             Algorithm    Purity  F-measure  Execution Time
0             k_median  0.753351   0.197849        0.023755
1            k_median2  0.753351   0.197849        0.608602
2  spectral_clustering  0.739946   0.740905        0.194289
3  k_median_polynomial  0.916890   0.108280        0.042042
4         k_median_rbf  0.753351   0.197849        0.046003


In [ ]:
########################## انجام الگوریتم ها در تکست هشتم(flame.txt)

In [ ]:

###################با جابه جایی یکی از مراکز با نقاط داده k_median

In [109]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link= ["http://cs.joensuu.fi/sipu/datasets/flame.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_media
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels
k=2
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
#زمان پابان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.85
F-measure: 0.1400029726977676
Execution time: 0.030186891555786133
Dataset with new labels:
     x1     x2  labels  new_labels
0  1.85  27.80       1           2
1  1.35  26.65       1           2
2  1.40  23.25       2           2
3  0.85  23.05       2           2
4  0.50  22.35       2           2


In [ ]:
###################### با جابه جایی دو تا از مراکز با نقاط داده k_median محاسبه

In [108]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/flame.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels
k =2
# زمان شروع
start_time = time.time()
centers, labels = k_median(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.85
F-measure: 0.1400029726977676
Execution time: 0.5584278106689453
Dataset with new labels:
     x1     x2  labels  new_labels
0  1.85  27.80       1           2
1  1.35  26.65       1           2
2  1.40  23.25       2           2
3  0.85  23.05       2           2
4  0.50  22.35       2           2


In [ ]:
#################با کرنل دو جمله ای k_median محاسبه

In [107]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import polynomial_kernel

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/flame.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تعریف تابع k_median_polynomial
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

k = 2
# زمان شروع
start_time = time.time()
centers, labels = k_median_polynomial(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

purity = purity_score(dataset)

# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)

    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

f_measure_value = f_measure(dataset)

# زمان پایان
end_time = time.time()
execution_time = end_time - start_time

print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.6375
F-measure: 0.3893129770992366
Execution time: 0.011703014373779297
Dataset with new labels:
     x1     x2  labels  new_labels
0  1.85  27.80       1           2
1  1.35  26.65       1           2
2  1.40  23.25       2           2
3  0.85  23.05       2           2
4  0.50  22.35       2           2


In [ ]:
#################با کرنل  گوسی k_median محاسبه

In [106]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.metrics.pairwise import rbf_kernel

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/flame.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)

# تعریف تابع k_median
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data.to_numpy(), centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

k = 2

# زمان شروع
start_time = time.time()
centers, labels = k_median_rbf(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels

# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

purity = purity_score(dataset)

# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = dataset["new_labels"].to_numpy()
        tp = np.sum((true_labels == i + 1) & (predicted_labels == i + 1))
        fp = np.sum((true_labels != i + 1) & (predicted_labels == i + 1))
        fn = np.sum((true_labels == i + 1) & (predicted_labels != i + 1))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

f_measure_value = f_measure(dataset)

# زمان پایان
end_time = time.time()
execution_time = end_time - start_time

print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.85
F-measure: 0.1400029726977676
Execution time: 0.02628326416015625
Dataset with new labels:
     x1     x2  labels  new_labels
0  1.85  27.80       1           2
1  1.35  26.65       1           2
2  1.40  23.25       2           2
3  0.85  23.05       2           2
4  0.50  22.35       2           2


In [ ]:
#################         الگوریتم خوشه بندی طیفی

In [105]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigsh
# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/flame.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
# تابع خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels
l=9
k =2
# زمان شروع
start_time = time.time()
labels = spectral_clustering(dataset[["x1", "x2"]], k)
labels += 1
dataset["new_labels"] = labels
# تعریف purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)
purity = purity_score(dataset)
# تعریف f_measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value
f_measure_value = f_measure(dataset)
# زمان پایان
end_time = time.time()
execution_time = end_time - start_time
print("Purity:", purity)
print("F-measure:", f_measure_value)
print("Execution time:", execution_time)
print("Dataset with new labels:")
print(dataset[["x1", "x2", "labels", "new_labels"]].head())

Purity: 0.6375
F-measure: 0.5555089400555338
Execution time: 0.1491241455078125
Dataset with new labels:
     x1     x2  labels  new_labels
0  1.85  27.80       1           1
1  1.35  26.65       1           1
2  1.40  23.25       2           1
3  0.85  23.05       2           1
4  0.50  22.35       2           1


In [104]:
import pandas as pd
import numpy as np
from collections import Counter
import requests
from io import StringIO
import time
from sklearn.neighbors import kneighbors_graph
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import polynomial_kernel, rbf_kernel
from scipy.sparse.linalg import eigsh

# بارگذاری داده‌ها
link = ["http://cs.joensuu.fi/sipu/datasets/flame.txt"]
numbers = []
read = requests.get(link[0])
content = read.text
data = StringIO(content)
df = pd.read_csv(data, sep="\s+", header=None, names=["x1", "x2", "labels"])
numbers.append(df)
dataset = pd.concat(numbers, ignore_index=True)
k=2
l=9

# تعریف تابع k_median
def k_median(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centers = np.array([np.median(data[labels == i].to_numpy(), axis=0) for i in range(k)])
        if np.all(centers == new_centers):
            break
        centers = new_centers
    return centers, labels

# نسخه دوم k_median (با فواصل مشابه نسخه اول شما)
def k_median2(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        distances = np.linalg.norm(data.to_numpy()[:, np.newaxis] - centers, axis=2)
        labels = np.argmin(distances, axis=1)
        for i in range(0, k, 2):
            if i < k and np.any(labels == i):
                centers[i] = data.iloc[labels == i].median(axis=0).to_numpy()
            if i + 1 < k and np.any(labels == (i + 1)):
                centers[i + 1] = data.iloc[labels == (i + 1)].median(axis=0).to_numpy()
        if np.allclose(centers, centers.mean(axis=0)):
            break
    return centers, labels

# K-Median با کرنل چندجمله‌ای
def k_median_polynomial(data, k, max_iterations=200):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = polynomial_kernel(data, centers)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = []
        for i in range(k):
            cluster_points = data.iloc[np.where(labels == i)]
            if len(cluster_points) > 0:
                new_centers.append(cluster_points.median(axis=0).to_numpy())
            else:
                new_centers.append(data.sample(n=1, random_state=42).to_numpy().flatten())
        new_centers = np.array(new_centers)
        if np.allclose(centers, new_centers, atol=1e-4):
            break
        centers = new_centers
    return centers, labels

# K-Median با کرنل گوسی
def k_median_rbf(data, k, max_iterations=100, gamma=0.5):
    centers = data.sample(n=k, random_state=42).to_numpy()
    for _ in range(max_iterations):
        similarity_matrix = rbf_kernel(data, centers, gamma=gamma)
        labels = np.argmax(similarity_matrix, axis=1)
        new_centers = np.array([data[labels == i].median(axis=0) for i in range(k)])
        if np.allclose(centers, new_centers):
            break
        centers = new_centers
    return centers, labels

# خوشه‌بندی طیفی
def spectral_clustering(data, k):
    adjacency_matrix = kneighbors_graph(data, n_neighbors=l, mode='connectivity', include_self=False).toarray()
    weight_matrix = adjacency_matrix * adjacency_matrix.T
    degree_matrix = np.diag(weight_matrix.sum(axis=1))
    laplacian_matrix = degree_matrix - weight_matrix
    eigvals, eigvecs = eigsh(laplacian_matrix, k=l, which='SM')
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(eigvecs)
    return labels

# محاسبه Purity
def purity_score(dataset):
    total = 0
    for i in range(k):
        cluster_labels = dataset[dataset["new_labels"] == (i + 1)]["labels"]
        if len(cluster_labels) > 0:
            most_common = Counter(cluster_labels).most_common(1)[0][1]
            total += most_common
    return total / len(dataset)

# محاسبه F-measure
def f_measure(dataset):
    precision_list = []
    recall_list = []
    for i in range(k):
        true_labels = dataset["labels"].to_numpy()
        predicted_labels = (dataset["new_labels"] == (i + 1)).to_numpy()
        tp = np.sum((true_labels[predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        fp = np.sum((true_labels[predicted_labels] != dataset["labels"].unique()[i]).astype(int))
        fn = np.sum((true_labels[~predicted_labels] == dataset["labels"].unique()[i]).astype(int))
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
    precision = np.mean(precision_list)
    recall = np.mean(recall_list)
    f_measure_value = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return f_measure_value

# اجرای الگوریتم‌ها
def run_algorithm(algorithm, dataset, k):
    start_time = time.time()
    if algorithm == spectral_clustering:
        labels = algorithm(dataset[["x1", "x2"]], k)
        centers = None
    else:
        centers, labels = algorithm(dataset[["x1", "x2"]], k)
    labels += 1
    dataset["new_labels"] = labels
    purity = purity_score(dataset)
    f_measure_value = f_measure(dataset)
    end_time = time.time()
    execution_time = end_time - start_time
    return purity, f_measure_value, execution_time, dataset

# نتایج
results = []
algorithms = [k_median, k_median2, spectral_clustering, k_median_polynomial, k_median_rbf]
for algorithm in algorithms:
    purity, f_measure_value, execution_time, dataset_with_labels = run_algorithm(algorithm, dataset.copy(), k)
    results.append({
        "Algorithm": algorithm.__name__,
        "Purity": purity,
        "F-measure": f_measure_value,
        "Execution Time": execution_time
    })

results_df = pd.DataFrame(results)

# مقایسه نتایج
print("مقایسه نتایج الگوریتم‌ها:")
print(results_df)

مقایسه نتایج الگوریتم‌ها:
             Algorithm    Purity  F-measure  Execution Time
0             k_median  0.850000   0.140003        0.041200
1            k_median2  0.850000   0.140003        0.512567
2  spectral_clustering  0.820833   0.803947        0.082193
3  k_median_polynomial  0.637500   0.389313        0.027694
4         k_median_rbf  0.850000   0.140003        0.070983


In [ ]:
#############purity , fmeasure,محاسبات نشان میدهد که از لحاظ معیارهای زمان اجرا
#نزدیکترین همسایه برای تشکیل گراف را خودمان مشخص کنیم .و این مورد منجر به بهبود عملکرد الگوریتم میشود بنیابراین میتوان گفت که عموما الگوریتم خوشه بندی طیفی در سناریوهای مختلف بهتر عمل میکندk الگوریتم خوشه بندی طیفی عملکرد بهتری دارد همچنین به

In [ ]:
####وقتی تعداد خوشه ها زیاد میشود زمان اجرای خوشه بندی طیفی زیاد میشود